In [ ]:
import random
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim



In [ ]:
# Generate unique full names
first_names = ["Ali", "Sara", "Reza", "Neda", "Hossein", "Leila", "Mojtaba", "Maryam", "Amir", "Zahra", "Farhad", "Elham"]
last_names = ["Ahmadi", "Hosseini", "Khan", "Jafari", "Karimi", "Rostami", "Salehi", "Soleimani", "Farhadi", "Akbari", "Shirazi", "Ghasemi"]
full_names = []
i = 24
while i > 0:
    full_name = f'{random.choice(first_names)} {random.choice(last_names)}'
    if full_name not in full_names:
        full_names.append(full_name)
        i -= 1
print(full_names)

['Sara Farhadi', 'Amir Jafari', 'Farhad Ghasemi', 'Reza Ghasemi', 'Hossein Rostami', 'Elham Ghasemi', 'Sara Rostami', 'Hossein Jafari', 'Sara Soleimani', 'Neda Hosseini', 'Mojtaba Karimi', 'Hossein Shirazi', 'Neda Rostami', 'Zahra Ghasemi', 'Elham Hosseini', 'Amir Soleimani', 'Elham Rostami', 'Reza Ahmadi', 'Leila Karimi', 'Leila Jafari', 'Amir Shirazi', 'Farhad Farhadi', 'Maryam Ghasemi', 'Elham Shirazi']


In [ ]:
# Generate conflict pairs
pairs_list = set()
matrix = np.zeros((24, 24), dtype=int)
while matrix.sum() < 40:
    num1 = np.random.choice(range(24))
    num2 = np.random.choice(range(24))
    if num1 == num2:
        continue
    pair = (num1, num2)
    if pair in pairs_list:
        continue
    pairs_list.add(pair)
    matrix[num1, num2] = 1
    matrix[num2, num1] = 1

conflicts = np.array([matrix])
print(conflicts)

[[[0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 1 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [0 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1]
  [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0]
  [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0]
  [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0

In [ ]:
# Create adjacent mask
def create_adjacent_mask(n_seats, seats_per_row, seats_per_col):
    adjacent_mask = np.zeros((n_seats, n_seats))
    for i in range(n_seats):
        if i % seats_per_row != 0:
            adjacent_mask[i, i-1] = 1
        if i % seats_per_row != seats_per_row-1:
            adjacent_mask[i, i+1] = 1
        if i >= seats_per_row:
            adjacent_mask[i, i-seats_per_row] = 1
        if i < n_seats-seats_per_row:
            adjacent_mask[i, i+seats_per_row] = 1
    return adjacent_mask

adjacent_mask = create_adjacent_mask(24, 6, 4)
print(adjacent_mask)

[[0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.

In [ ]:

# PyTorch model and training
class SeatingArr(nn.Module):
    def __init__(self) -> None:
        super(SeatingArr, self).__init__()
        self.flatten = nn.Flatten()
        self.dense_1 = nn.Linear(24*24, 80, dtype=torch.float64)
        self.dropout_1 = nn.Dropout(0.5)
        self.dense_2 = nn.Linear(80, 24 * 24, dtype=torch.float64)
        self.dropout_2 = nn.Dropout(0.5)

    def forward(self, x):
        x = self.flatten(x)
        x = F.relu(self.dense_1(x))
        x = self.dropout_1(x)
        x = self.dense_2(x)
        x = x.view(-1, 24, 24)
        x = F.softmax(x, dim=2)
        return x


In [ ]:
net = SeatingArr()
optimizer = optim.Adam(params=net.parameters(), lr=0.001)  # Further reduced learning rate
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.1)  # Dynamic learning rate
conflicts_tensor = torch.tensor(conflicts, dtype=torch.float64)
adjacent_mask_torch = torch.tensor(adjacent_mask, dtype=torch.float64)

In [ ]:
# Function to calculate conflicts
def calculate_conflict_torch(seating_arrangement, conflict_matrix):
    ca_mul = conflict_matrix * adjacent_mask_torch
    conflicts = torch.sum(torch.matmul(seating_arrangement.type(torch.float64), ca_mul))
    return conflicts

In [ ]:
# Custom loss function
def custom_loss_torch(predicted_seating_arrangement, conflicts_tensor):
    loss = 0
    batch_size = predicted_seating_arrangement.shape[0]
    predicted_seating_arrangement = predicted_seating_arrangement.type(torch.float64)
    conflict = calculate_conflict_torch(predicted_seating_arrangement, conflicts_tensor)
    col_sum_loss = torch.sum((torch.sum(predicted_seating_arrangement, dim=1) - 1) ** 2)
    loss = col_sum_loss + conflict
    return loss / batch_size

In [ ]:
# Function to print seating arrangement
def print_seating_arrangement(epoch, output, phase):
    seating_arrangement = torch.argmax(output, dim=2).numpy()
    for i in range(seating_arrangement.shape[0]):
        names_arrangement = [full_names[j] for j in seating_arrangement[i]]
        print(f'{phase} Epoch {epoch + 1} - Arrangement {i + 1}: {names_arrangement}')
        print(f'{phase} Epoch {epoch + 1} - Arrangement {i + 1} (Indices): {seating_arrangement[i]}')


In [ ]:
# Training function
def train(epoch):
    net.train()
    optimizer.zero_grad()
    output = net(conflicts_tensor)
    loss = custom_loss_torch(output, conflicts_tensor)
    loss.backward()
    optimizer.step()
    scheduler.step()
    if epoch % 10 == 0:
        print(f'Train Epoch: {epoch + 1} Loss: {loss.item()}')
        torch.save(net.state_dict(), 'model.pth')
        torch.save(optimizer.state_dict(), 'optimizer.pth')
        print_seating_arrangement(epoch, output, "Intermediate")


In [ ]:
# Initial output
with torch.no_grad():
    initial_output = net(conflicts_tensor)
    print_seating_arrangement(0, initial_output, "Initial")

# Training loop
n_epoch = 400
for epoch in range(n_epoch):
    train(epoch)

# Final output
with torch.no_grad():
    final_output = net(conflicts_tensor)
    final_loss = custom_loss_torch(final_output, conflicts_tensor)
    print(f'Final Loss: {final_loss.item()}')
    print_seating_arrangement(n_epoch, final_output, "Final")

Initial Epoch 1 - Arrangement 1: ['Elham Ghasemi', 'Reza Ahmadi', 'Sara Soleimani', 'Elham Shirazi', 'Neda Rostami', 'Reza Ahmadi', 'Elham Hosseini', 'Amir Soleimani', 'Hossein Rostami', 'Hossein Shirazi', 'Sara Rostami', 'Neda Hosseini', 'Mojtaba Karimi', 'Hossein Rostami', 'Sara Farhadi', 'Neda Hosseini', 'Elham Shirazi', 'Amir Soleimani', 'Reza Ahmadi', 'Reza Ahmadi', 'Sara Soleimani', 'Sara Rostami', 'Sara Soleimani', 'Elham Hosseini']
Initial Epoch 1 - Arrangement 1 (Indices): [ 5 17  8 23 12 17 14 15  4 11  6  9 10  4  0  9 23 15 17 17  8  6  8 14]
Train Epoch: 1 Loss: 4.501451382135095
Intermediate Epoch 1 - Arrangement 1: ['Hossein Shirazi', 'Elham Rostami', 'Neda Hosseini', 'Elham Shirazi', 'Elham Hosseini', 'Elham Hosseini', 'Sara Rostami', 'Elham Hosseini', 'Reza Ghasemi', 'Hossein Shirazi', 'Zahra Ghasemi', 'Mojtaba Karimi', 'Reza Ghasemi', 'Neda Rostami', 'Leila Karimi', 'Reza Ghasemi', 'Elham Ghasemi', 'Neda Rostami', 'Hossein Rostami', 'Sara Soleimani', 'Sara Farhadi', '